In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [2]:
df = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_meta_cleaned.csv", engine= "python", encoding='utf-8')
df.head()

,index,movie_id,title,release_year,runtime,mpa_rating,mpa_rating_origin,imdb_score,dvd_sales,blu_sales,...,kwrds,actor_1,actor_2,actor_3,actor_4,income_opening,votes,income_usa,income_int,income_ww
0,0,tt4154796,Avengers: Endgame,2019,181,PG-13,.,8.5,19581271,69437591,...,\n time travel\n \n superhero\n ...,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,357115007,661719,858373000,1939427564,2797800564
1,1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,136,PG-13,.,6.6,35478573,58936309,...,\n pirate\n \n captain\n \n ...,Johnny Depp,Penélope Cruz,Geoffrey Rush,Ian McShane,90151958,460865,241063875,804600000,1045663875
2,2,tt2395427,Avengers: Age of Ultron,2015,141,PG-13,.,7.3,33569045,55802159,...,\n superhero\n \n marvel cinemati...,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,191271109,697604,459005868,944008095,1403013963
3,3,tt4154756,Avengers: Infinity War,2018,156,PG-13,.,8.5,23700665,77736363,...,\n death of recurring character\n \n ...,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,257698183,748996,678815482,1369544272,2048359754
4,4,tt0449088,Pirates of the Caribbean: At World's End,2007,167,PG-13,.,7.1,314040375,7640482,...,\n mass hanging\n \n pirate\n ...,Johnny Depp,Geoffrey Rush,Orlando Bloom,Keira Knightley,114732820,572118,309420425,654000000,963420425


In [4]:
# string to numeric
col_numeric = ['release_year','runtime','imdb_score','dvd_sales','blu_sales','total_sales',
            'legs','share','inf_income_usa','theater_opening','theater_total',
            'metascore','big_awards_num','awards_win_num','awards_nomin_num',
            'reviews_users','reviews_critics','budget','series_new','income_opening',
            'votes','income_usa','income_int','income_ww']
for col in col_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.replace(np.nan, '.', regex=True)

In [5]:
df['runtime'][1]

136.0

In [4]:
# 불필요한 column 제거
df = df.drop(['index','mpa_rating_origin'],axis=1)

In [5]:
# NAN -> '.'
for col in df.columns:
        df.loc[df[col].isna(), col] = '.'

In [6]:
# 불필요한 row 제거


In [7]:
# inflation 파생변수
df['inf'] = '.'
for i in df.index:
        if (df.loc[i,'inf_income_usa'] != '.') & (df.loc[i,'income_usa'] != '.') & (df.loc[i,'income_usa'] != 0):
            df.loc[i,'inf'] = float(df.loc[i,'inf_income_usa']) / float(df.loc[i,'income_usa'])
        else:
            pass

In [8]:
df['inf']

0             1
1        1.1488
2       1.08066
3             1
4       1.32413
         ...   
5530    1.01561
5531          1
5532    1.01283
5533     1.0156
5534          .
Name: inf, Length: 5535, dtype: object

In [9]:
# genre 더미변수
# scraping 오류 제거
df.loc[df['movie_id'] == 'tt5851562', 'genre'] = 'Sci-Fi'
df.loc[df['movie_id'] == 'tt1781967', 'genre'] = 'Game-Show'
df.loc[df['movie_id'] == 'tt7048386', 'genre'] = 'Crime, Drama, Horror, Mystery, Thriller'
df.loc[df['movie_id'] == 'tt0498879', 'genre'] = 'Reality-TV'
df.loc[df['movie_id'] == 'tt4403432', 'genre'] = 'Crime'
df.loc[df['movie_id'] == 'tt2010510', 'genre'] = 'Reality-TV'
df.loc[df['movie_id'] == 'tt4950016', 'genre'] = 'Comedy'



In [10]:
# 전체 장르 목록 구하기
genre_list = []
for i in df.index:
    genre = df.loc[i,'genre'].split(',')
    for g in genre:
        g = g.strip().lower()
        if g not in genre_list:
            genre_list.append(g)
genre_list.remove('.')

In [11]:
genre_list

['action',
 'adventure',
 'drama',
 'sci-fi',
 'fantasy',
 'thriller',
 'animation',
 'family',
 'musical',
 'comedy',
 'romance',
 'mystery',
 'war',
 'western',
 'history',
 'sport',
 'horror',
 'crime',
 'music',
 'biography',
 'documentary',
 'short',
 'game-show',
 'adult',
 'news',
 'fiction',
 'film-noir',
 'reality-tv']

In [12]:
# 각 장르별 더미 변수 만들기
for i in df.index:
    genre_0 = df.loc[i,'genre'].split(',')
    genre_1 = []
    for g in genre_0:
        g = g.strip().lower()
        genre_1.append(g)
    for g in genre_list:
        if g in genre_1:
            df.loc[i,'genre_{}'.format(g)] = 1
        else:
            df.loc[i,'genre_{}'.format(g)] = 0

In [15]:
len(df.columns)

81

In [14]:
df[df['genre_action'] == 1]

,movie_id,title,release_year,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,legs,...,genre_music,genre_biography,genre_documentary,genre_short,genre_game-show,genre_adult,genre_news,genre_fiction,genre_film-noir,genre_reality-tv
0,tt4154796,Avengers: Endgame,2019,181,PG-13,8.5,1.95813e+07,6.94376e+07,8.90189e+07,2.40363,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,136,PG-13,6.6,3.54786e+07,5.89363e+07,9.44149e+07,2.67397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,tt2395427,Avengers: Age of Ultron,2015,141,PG-13,7.3,3.3569e+07,5.58022e+07,8.93712e+07,2.39977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tt4154756,Avengers: Infinity War,2018,156,PG-13,8.5,2.37007e+07,7.77364e+07,1.01437e+08,2.63415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,tt0449088,Pirates of the Caribbean: At World's End,2007,167,PG-13,7.1,3.1404e+08,7.64048e+06,3.21681e+08,2.69688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,tt7431594,Race 3,2018,160,NOT RATED,2,.,.,.,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5527,tt6836936,Saaho,2019,180,NOT RATED,5.3,.,.,.,1.54251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5531,tt5970844,Thugs of Hindostan,2018,164,NOT RATED,4,.,.,.,1.2231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5532,tt5956100,Tiger Zinda Hai,2017,145,NOT RATED,6,.,.,.,3.11169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [204]:
df['genre'][385].split(',')

list

In [98]:
df['genre']

0       Action, Adventure, Drama, Sci-Fi
1             Action, Adventure, Fantasy
2              Action, Adventure, Sci-Fi
3              Action, Adventure, Sci-Fi
4             Action, Adventure, Fantasy
                      ...               
5530              Comedy, Drama, Romance
5531                   Action, Adventure
5532         Action, Adventure, Thriller
5533                          Drama, War
5534                Action, Crime, Drama
Name: genre, Length: 5535, dtype: object

In [24]:
df['country']

0           USA
1       USA, UK
2           USA
3           USA
4           USA
         ...   
5530      India
5531      India
5532      India
5533      India
5534      India
Name: country, Length: 5535, dtype: object

In [26]:
for i in df.index:
    country_list = df.loc[i,'country'].split(',')
    

In [28]:
df['country_1'] = df['country'].str.split(',').str[0]
df['country_2'] = df['country'].str.split(',').str[1]
df['country_3'] = df['country'].str.split(',').str[2]
df['country_1'].str.strip()
df['country_2'].str.strip()
df['country_3'].str.strip()

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
5530    NaN
5531    NaN
5532    NaN
5533    NaN
5534    NaN
Name: country_3, Length: 5535, dtype: object

In [35]:
df['country_3'].head(10)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5          UK
6     Austria
7         NaN
8         NaN
9         NaN
Name: country_3, dtype: object

In [34]:
df['country'].head(10)

0                                         USA
1                                     USA, UK
2                                         USA
3                                         USA
4                                         USA
5                             USA, Canada, UK
6    UK, USA, Austria, Mexico, Italy, Morocco
7                                         USA
8                                         USA
9                                         USA
Name: country, dtype: object